In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
pwd


'C:\\Users\\Paxcom\\Downloads'

In [3]:
cd C:\Users\Paxcom\Downloads

C:\Users\Paxcom\Downloads


In [7]:
dc_listings = pd.read_csv('listings.csv.gz')
dc_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [8]:
dc_listings = dc_listings[['accommodates', 'bedrooms', 'bathrooms','number_of_reviews','price']]

In [9]:
dc_listings.head()

,accommodates,bedrooms,bathrooms,number_of_reviews,price
0,4,1.0,1.0,0,$160.00
1,6,3.0,3.0,65,$350.00
2,1,1.0,2.0,1,$50.00
3,2,1.0,1.0,0,$95.00
4,4,1.0,1.0,0,$50.00


In [10]:
dc_listings['price'].head()

0    $160.00
1    $350.00
2     $50.00
3     $95.00
4     $50.00
Name: price, dtype: object

In [11]:
dc_listings['price'] = dc_listings['price'].apply(lambda x:x.replace('$',''))
dc_listings['price'] = dc_listings['price'].apply(lambda x:x.replace(',',''))
dc_listings['price'] = dc_listings['price'].astype('float')

In [12]:
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723 entries, 0 to 3722
Data columns (total 5 columns):
accommodates         3723 non-null int64
bedrooms             3702 non-null float64
bathrooms            3696 non-null float64
number_of_reviews    3723 non-null int64
price                3723 non-null float64
dtypes: float64(3), int64(2)
memory usage: 145.5 KB


In [13]:
dc_listings = dc_listings.dropna(axis=0)
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3675 entries, 0 to 3722
Data columns (total 5 columns):
accommodates         3675 non-null int64
bedrooms             3675 non-null float64
bathrooms            3675 non-null float64
number_of_reviews    3675 non-null int64
price                3675 non-null float64
dtypes: float64(3), int64(2)
memory usage: 172.3 KB


In [14]:
train_rows  =  int(dc_listings.shape[0] * 0.7)
train = dc_listings.iloc[0:train_rows]
test = dc_listings.iloc[train_rows:]

In [15]:
features = list(train.columns)


In [16]:
def predict_price(row,temp_df):
    distances = temp_df[features] - row
    squared = np.square(distances)
    dist = squared.sum(axis=1)
    temp_df['distance'] = dist**0.5
    new_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = new_df['price'].iloc[0:5]
    predicted_price = nearest_neighbor_prices.mean()
    return predicted_price
   

In [17]:
for i,row in test[features].iterrows():
    predicted_price = predict_price(row,train)
    test.loc[i,'predicted_price'] = predicted_price

In [18]:
test.predicted_price.head(10)

2615    500.0
2616    399.8
2617    284.0
2618    110.0
2619     99.6
2620    125.0
2621     55.0
2622    120.0
2623    129.6
2624     70.2
Name: predicted_price, dtype: float64

In [22]:
mae = np.absolute(test['price'] - test['predicted_price']).sum()/test.shape[0]
print(mae)

1.9260199456029008
